<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/TMLC_LLM_projects/Finetune_OpenAI_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Install the required library

Make sure you have the `openai` library installed. Use the command below if not already installed:

In [1]:
!pip install openai==1.55.3 httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [2]:
# Import necessary libraries
import os
import pandas as pd
import json
from openai import OpenAI

Access data from this [link](https://themlco-my.sharepoint.com/:f:/p/chiragchauhan/Eg7biBxP1hhEta6JLchrfWgBvLawQsfxSDJO6K1BQl_J9g?e=FhiWT3)

## Step 2: Set up your API key
Replace 'YOUR_API_KEY' with your actual OpenAI API key.

In [3]:
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_KEY'

## Step 3: Prepare the dataset

Hint: Use a JSON or CSV file and convert it to JSONL

If loading a CSV dataset and convert it to JSONL format.
Complete the conversion code below:

In [8]:
## JSONL file creation
import pandas as pd
import json

df = pd.read_csv('/content/GenAI_Program_FAQ_dataset.csv')

In [9]:
df.head()

,prompt,completion
0,How the sessions will be organized. Will the r...,This program contains both pre-recorded and li...
1,Is there any pre-requisite required before or ...,Intermediate level python programming is requi...
2,What will be the duration of the program?,This is a 6 weeks long program.
3,What materials will be provided during the cou...,"Videos, documents, coding assignments will be ..."
4,What will be the timings of the sessions?,Live sessions will happen 6 to 7:30 PM IST on ...


In [13]:
with open("faq_data.jsonl", "w") as f:
  for _, row in df.iterrows():
    json_line = {
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['completion']}
        ]
        }
    f.write(json.dumps(json_line) + "\n")

## Step 4: Upload the file for fine-tuning

Use the OpenAI API to upload the dataset. Replace '<JSONL_FILE>' with your dataset file name.

In [14]:
# upload file with openai client.files
client = OpenAI()
client.files.create(
    file=open("faq_data.jsonl", "rb"),
    purpose='fine-tune'
)

FileObject(id='file-JhKcwRpSxggRkMxgZ8ka5i', bytes=8409, created_at=1733667887, filename='faq_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

## Step 5: Fine-tune the model

Trigger the fine-tuning job process using the uploaded file ID. Replace '<FILE_ID>' and '<MODEL_NAME>' accordingly.

In [16]:
# Use client.fine_tuning.jobs.create
client.fine_tuning.jobs.create(
    training_file="file-JhKcwRpSxggRkMxgZ8ka5i",
    model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-47oADAFHIraKWFj3MIVzC29w', created_at=1733667943, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-bQVIhPzVjnoakAjRyRFBoo5Y', result_files=[], seed=1289404331, status='validating_files', trained_tokens=None, training_file='file-JhKcwRpSxggRkMxgZ8ka5i', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

## Step 6: Monitor and use the fine-tuned model

Check list of fine-tuning jobs, retrieve job details.

In [17]:
# fine_tuning.jobs.list
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-47oADAFHIraKWFj3MIVzC29w', created_at=1733667943, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-bQVIhPzVjnoakAjRyRFBoo5Y', result_files=[], seed=1289404331, status='validating_files', trained_tokens=None, training_file='file-JhKcwRpSxggRkMxgZ8ka5i', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None), FineTuningJob(id='ftjob-piheNt0g58nraC0gv3zG7Cvb', created_at=1733662110, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal:faq-model:AcBAxlr4', finished_at=1733662785, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-bQVIhPzVjnoak

In [18]:
# Completion calls can be added to test the fine-tuned model.
client.fine_tuning.jobs.retrieve("ftjob-47oADAFHIraKWFj3MIVzC29w")
# Remember to test and validate the model after fine-tuning!

FineTuningJob(id='ftjob-47oADAFHIraKWFj3MIVzC29w', created_at=1733667943, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-bQVIhPzVjnoakAjRyRFBoo5Y', result_files=[], seed=1289404331, status='running', trained_tokens=None, training_file='file-JhKcwRpSxggRkMxgZ8ka5i', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [24]:
client.fine_tuning.jobs.list_events(
    fine_tuning_job_id="ftjob-47oADAFHIraKWFj3MIVzC29w", limit=50
).data

[FineTuningJobEvent(id='ftevent-AbhOWUBOzWpfcORWmHTfPCTT', created_at=1733668299, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-wdc44vdFyz0gXmi152ebSCD8', created_at=1733668296, level='info', message='New fine-tuned model created', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-4nCxOJPHlUqP4ILbOCrL32bN', created_at=1733668296, level='info', message='Checkpoint created at step 56', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-Kl224QV5l71FCjVdfBc3VXr3', created_at=1733668296, level='info', message='Checkpoint created at step 28', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-uGPw7yCYe9QOIk3kQxWahqG7', created_at=1733668276, level='info', message='Step 84/84: training loss=0.60', object='fine_tuning.job.event', data={'step': 84, 'train_loss': 0.604470252990

## Step 7: Use the fine-tuned model

In [4]:
from openai import OpenAI
client = OpenAI()

In [23]:
completion = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal::AcCbn4dP",
    messages=[
        {"role": "system", "content": "You are a helpful assistant to answering TMLC questions."},
        {"role": "user", "content": "Can you tell what's the duration of TMLC course?"}
    ]
)
print("Fine-tuned model response:", completion.choices[0].message.content)

Fine-tuned model response: TMLC is a 1-month Comprehensive course.
